In [1]:
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import usefulFunctions.exergyFunctions as ef
import usefulFunctions.thermoFunctions as tf

In [2]:
gasconst = PropsSI('gas_constant','T',298.15,'P',101325,'PR::H2') # getting the gas constant from coolprop

In [3]:
chemExergyPath = 'D:/googledrive/DLR works/Chem exergies.csv' # database of chemical exergies

In [4]:
ech_df = pd.read_csv(chemExergyPath, delimiter=';', index_col=0)

In [5]:
ech_df

,State,Standard Chem Exergy (kJ/mol)
Al,s,795.7
Al4C3,s,4216.2
AlCl3,s,352.2
Al2O3,s. α corundum,15
Al2O3•H2O,s. boermite,9.4
...,...,...
ZnO,s,22.9
Zn(OH)2,s. β,25.7
ZnS,s. sphalerite,747.6
ZnSO4,s,82.3


In [6]:
def ech_stream(x_input, c_input):
    """
    Function for calculating molar chemical exergy of a gas mixture 
    based on the chosen chemical exergy database
    
    x_input: list of molar fractions of the species in the stream
    c_input: list of species names (example: O2, CO2) as a string
    
    returns chemical exergy in J/s
    """
    Tenv = 298.15 # K
    Penv = 100000 # Pa
    x_0 = np.zeros(len(c_input))
    Ex_ch_c = np.zeros(len(c_input))
    x_0_H2O = (0.03169/1) # from Raoult's law
    
    # flash calculation for water
    if 'H2O' in c_input:
        H2O = c_input.index('H2O')
        if x_input[H2O] <= x_0_H2O:
            rvapor = 1
            x_H2Obase = x_input[H2O]
        else:
            x_H2Obase = x_0_H2O
            rvapor = ((1-x_input[H2O])/(1-x_H2Obase)) # ratio of the part of the stream that is in gas/vapour at dead state
        for c in range(len(c_input)):
            if c_input[c] != 'H2O':
                x_0[c] = x_input[c]*(1-x_H2Obase)/(1-(x_input[H2O]))
            elif x_input[c]==0:
                x_0[c] = 0
            else:
                x_0[c] = x_H2Obase # saturated water vapour
                
        rliquid = (1-rvapor) # ratio of stream that condenses at dead state
        r_H2O = rliquid + rvapor*x_0[H2O]
        
        # chemical exergy calculation
        for c in range(len(c_input)):
            species = c_input[c]
            if x_input[c] == 0:
                Ex_ch_c[c] = 0
            else:
                if species!='H2O':
                    Ex_ch_c[c] = float(ech_df.loc[species]['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c])
                else:
                    Ex_ch_c[c] = (x_0[c]*rvapor*(float(ech_df.loc['H2O']['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c]))) + rliquid*float(ech_df.loc['H2O (L)']['Standard Chem Exergy (kJ/mol)'])*1000/r_H2O
    else:
        x_0 = x_input
        for c in range(len(c_input)):
            species = c_input[c]
            if x_input[c] == 0:
                Ex_ch_c[c] = 0
            else:
                Ex_ch_c[c] = float(ech_df.loc[species]['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c])
    
    Ex_ch = np.sum(x_input*Ex_ch_c)
    
    return Ex_ch
        
    # use float(ech_df.loc['O2']['Standard Chem Exergy (kJ/mol)']) to obtain value

In [7]:
# calculates exergy of stream using coolprop

def e_CP(name): # input stream name as string with ''
    """
    Function for calculating total exergy of a stream using CoolProp
    
    name: name of stream
    
    returns total exergy of a stream in J/s
    """
    species_list = []
    xlist = []
    for k in range(nSpecies):
        C = d[name]['C'+str(k+1)]
        c_short = spec[C]
        species_list.append((c_short))
        x_stream = float(d[name]['Molar Fraction (Vapor) - '+C+' /-'])
        xlist.append((x_stream))
    m_stream = float(d[name]['Molar Flow /kmol/h']) * (1000/3600) # mass flow (mol/s)
    ephlist = []
    hlist = []
    slist = []
    for k in range(nSpecies):
        h_c = xlist[k]*PropsSI('Hmolar','T|gas',float(d[name]['Temperature /C'])+Tenv,'P',float(d[name]['Pressure /bar'])*1e5,'PR::'+spec[d[name]['C'+str(k+1)]])
        henv_c = xlist[k]*PropsSI('Hmolar','T|gas',Tenv,'P',Penv,'PR::'+spec[d[name]['C'+str(k+1)]])
        s_c = xlist[k]*PropsSI('Smolar','T|gas',float(d[name]['Temperature /C'])+Tenv,'P',float(d[name]['Pressure /bar'])*1e5,'PR::'+spec[d[name]['C'+str(k+1)]])
        senv_c = xlist[k]*PropsSI('Smolar','T|gas',Tenv,'P',Penv,'PR::'+spec[d[name]['C'+str(k+1)]])
        h = h_c - henv_c # enthalpy difference between system and environment
        hlist.append((h))
        s = s_c - senv_c
        slist.append((s))
    EPHCP = sum(hlist) - Tenv*sum(slist) # calculating physical exergy
    ECH = ech_stream(xlist,species_list) # calculating chemical exergy using ech_stream function
    Etot = (ECH + EPHCP) * m_stream
    return Etot



In [14]:
filePath = "D:/googledrive/DLR works/SOC heater recuperator but xml.xml"
# don't forget to change the encoding of the xml file into utf-8

In [15]:
tree = ET.parse(source =filePath) # Get ElementTree
root = tree.getroot() # Extract Data
root # Show

<Element 'Objects' at 0x000002F13DDC7C70>

In [16]:
objects = root.findall("Object") #Extract Objects into a list
objects

[<Element 'Object' at 0x000002F143DC09A0>,
 <Element 'Object' at 0x000002F143ED4630>,
 <Element 'Object' at 0x000002F143EE04A0>,
 <Element 'Object' at 0x000002F143EE7860>,
 <Element 'Object' at 0x000002F143EE3B80>,
 <Element 'Object' at 0x000002F1440D5040>,
 <Element 'Object' at 0x000002F1440DB400>,
 <Element 'Object' at 0x000002F1440E17C0>,
 <Element 'Object' at 0x000002F1440E8B80>,
 <Element 'Object' at 0x000002F1440EEF40>,
 <Element 'Object' at 0x000002F1440FA360>,
 <Element 'Object' at 0x000002F144100720>,
 <Element 'Object' at 0x000002F144106AE0>,
 <Element 'Object' at 0x000002F14410DEA0>,
 <Element 'Object' at 0x000002F14411A2C0>,
 <Element 'Object' at 0x000002F144120680>,
 <Element 'Object' at 0x000002F144126A40>,
 <Element 'Object' at 0x000002F144126AE0>,
 <Element 'Object' at 0x000002F144126B80>,
 <Element 'Object' at 0x000002F144126C20>,
 <Element 'Object' at 0x000002F144126CC0>,
 <Element 'Object' at 0x000002F14412E270>,
 <Element 'Object' at 0x000002F14412E7C0>,
 <Element '

In [17]:
test2 = objects[0].findall("Property") # Extract properties of first object into a list
test2

[<Element 'Property' at 0x000002F143F06090>,
 <Element 'Property' at 0x000002F143F06220>,
 <Element 'Property' at 0x000002F143F06450>,
 <Element 'Property' at 0x000002F143F063B0>,
 <Element 'Property' at 0x000002F143F06310>,
 <Element 'Property' at 0x000002F143F06130>,
 <Element 'Property' at 0x000002F143F061D0>,
 <Element 'Property' at 0x000002F143F060E0>,
 <Element 'Property' at 0x000002F143F06630>,
 <Element 'Property' at 0x000002F143F065E0>,
 <Element 'Property' at 0x000002F143ED7310>,
 <Element 'Property' at 0x000002F143ED7450>,
 <Element 'Property' at 0x000002F143ED7360>,
 <Element 'Property' at 0x000002F143ED7540>,
 <Element 'Property' at 0x000002F143ED75E0>,
 <Element 'Property' at 0x000002F143ED7590>,
 <Element 'Property' at 0x000002F143ED7720>,
 <Element 'Property' at 0x000002F143ED77C0>,
 <Element 'Property' at 0x000002F143ED7810>,
 <Element 'Property' at 0x000002F143ED7770>,
 <Element 'Property' at 0x000002F143ED78B0>,
 <Element 'Property' at 0x000002F143ED7900>,
 <Element 

In [18]:
# dictionary for short and long names of species
spec = {'Nitrogen':'N2', 'Hydrogen':'H2', 'Oxygen':'O2', 'Water':'H2O', 'Carbon Dioxide':'CO2', 'Carbon Monoxide':'CO', 'Ammonia':'NH3', 'Methane':'CH4'}

In [19]:
# xml extraction + stream exergy calculation 

tree = ET.parse(source =filePath) # Get ElementTree
root = tree.getroot() # Extract Data
objects = root.findall("Object") #Extract Objects into a list

Tenv = 298.15 # K
Penv = 100000 # Pa, can use 101325?
d = {} # put results into this dictionary
st = {} # dict for stream names
nSpecies = 4 # number of species (CHANGE THIS MANUALLY ACCORDING TO THE SYSTEM)
a=0

for i in range(len(objects)):

    print(objects[i].attrib) # Print attributes of the object: name and type
    props = objects[i].findall("Property") # Get list of properties for each object
    d[objects[i].attrib['name']] = {"type": objects[i].attrib['type']}
    p = len(props)
    j=0
        
    if 'Material Stream' in objects[i].attrib['type']:
        st_key = 'st_'+str(a+1)
        st_val = objects[i].attrib['name']
        st[st_key] = st_val
        a+=1
    
    while j <p:
        if props[j].attrib['units']=='': #Change units
            props[j].attrib['units']='-'

        if 'Molar Fraction' in props[j].attrib['name']:
            print('----------')
            for k in range(nSpecies):
                print(str(j+k+1) + ' -> ' + props[j].attrib['name'] + ' - ' + props[j+k+1].attrib['name'] + ' /' + props[j].attrib['units'] + ': ' + props[j+k+1].attrib['value'])
                key = props[j].attrib['name'] + ' - ' + props[j+k+1].attrib['name'] + ' /' + props[j].attrib['units']
                val = props[j+k+1].attrib['value']
                d[objects[i].attrib['name']][key] = val
            print('----------')
            j+=nSpecies+1
        else:
            print(str(j) + ' -> ' + props[j].attrib['name'] + ' /' + props[j].attrib['units'] + ': ' + props[j].attrib['value'])
            key = props[j].attrib['name'] + ' /' + props[j].attrib['units']
            val = props[j].attrib['value']
            d[objects[i].attrib['name']][key] = val
            j+=1
    
    if objects[i].attrib['type'] == 'Material Stream':
        # extracting names of components in the material streams
        for k in range(nSpecies):
            print(str(p+k) + ' -> Species ' + str(k+1) + ': ' + props[6+k].attrib['name'])
            key = 'C'+str(k+1)
            val = props[6+k].attrib['name']
            d[objects[i].attrib['name']][key] = val
        print('----------')    
        species_list = []
        xlist = []
        for k in range(nSpecies):
            C = d[objects[i].attrib['name']]['C'+str(k+1)]
            c_short = spec[C]
            species_list.append((c_short))
            x_stream = float(d[objects[i].attrib['name']]['Molar Fraction (Vapor) - '+C+' /-'])
            xlist.append((x_stream))
        # calculation of exergy of each streams using the available simulation data
        m_stream = float(d[objects[i].attrib['name']]['Molar Flow /mol/s']) # mass flow (mol/s)
        h_stream = float(d[objects[i].attrib['name']]['Molar Enthalpy (Mixture) /kJ/kmol'])
        s_stream = float(d[objects[i].attrib['name']]['Molar Entropy (Mixture) /kJ/[kmol.K]'])
        ECH = ech_stream(xlist,species_list) # J/mol
        EPH = h_stream - Tenv*s_stream # J/mol
        E_stream = (ECH + EPH) * m_stream # J/s
        print(str(p+k+1) + ' -> Exergy of Stream /J/s: ' + str(E_stream))
        print('----------')
        key = 'Exergy of Stream /J/s'
        val = str(E_stream)
        d[objects[i].attrib['name']][key] = val    
        
    print()

{'name': 'MSTR-12', 'type': 'Material Stream'}
0 -> Temperature /K: 1073.15
1 -> Pressure /Pa: 101325
2 -> Mass Flow /kg/s: 0.991709
3 -> Molar Flow /mol/s: 34.4025
4 -> Volumetric Flow /m3/s: 3.02931
----------
6 -> Mixture Molar Fraction - Hydrogen /-: 0
7 -> Mixture Molar Fraction - Oxygen /-: 0.204052
8 -> Mixture Molar Fraction - Water /-: 0
9 -> Mixture Molar Fraction - Nitrogen /-: 0.795948
----------
10 -> Density (Mixture) /kg/m3: 0.327371
11 -> Molecular Weight (Mixture) /kg/kmol: 28.8266
12 -> Specific Enthalpy (Mixture) /kJ/kg: 837.841
13 -> Specific Entropy (Mixture) /kJ/[kg.K]: 1.50151
14 -> Molar Enthalpy (Mixture) /kJ/kmol: 24152.1
15 -> Molar Entropy (Mixture) /kJ/[kmol.K]: 43.2835
16 -> Thermal Conductivity (Mixture) /W/[m.K]: 0.0727356
----------
18 -> Molar Fraction (Vapor) - Hydrogen /-: 0
19 -> Molar Fraction (Vapor) - Oxygen /-: 0.204052
20 -> Molar Fraction (Vapor) - Water /-: 0
21 -> Molar Fraction (Vapor) - Nitrogen /-: 0.795948
----------
----------
23 -> Mol

In [20]:
print(st)

{'st_1': 'MSTR-12', 'st_2': 'MSTR-11', 'st_3': 'MSTR-10', 'st_4': 'MSTR-09', 'st_5': 'MSTR-08', 'st_6': 'MSTR-07', 'st_7': 'air_out', 'st_8': 'water_out', 'st_9': 'air_in', 'st_10': 'water_in', 'st_11': 'MSTR-05', 'st_12': 'MSTR-06', 'st_13': 'MSTR-13', 'st_14': 'MSTR-14', 'st_15': 'MSTR-15', 'st_16': 'MSTR-16'}


In [21]:
d.keys()

dict_keys(['MSTR-12', 'MSTR-11', 'MSTR-10', 'MSTR-09', 'MSTR-08', 'MSTR-07', 'air_out', 'water_out', 'air_in', 'water_in', 'MSTR-05', 'MSTR-06', 'MSTR-13', 'MSTR-14', 'MSTR-15', 'MSTR-16', 'ESTR-02', 'ESTR-01', 'ESTR-03', 'ESTR-04', 'HEAT-01', 'HEAT-02', 'HE-01', 'HE-02', 'HE-03', 'HE-04'])

In [22]:
df = pd.DataFrame.from_dict(d)

In [23]:
df

,MSTR-12,MSTR-11,MSTR-10,MSTR-09,MSTR-08,MSTR-07,air_out,water_out,air_in,water_in,...,ESTR-02,ESTR-01,ESTR-03,ESTR-04,HEAT-01,HEAT-02,HE-01,HE-02,HE-03,HE-04
type,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,Material Stream,...,Energy Stream,Energy Stream,Energy Stream,Energy Stream,Stream Heater,Stream Heater,Heat Exchanger,Heat Exchanger,Heat Exchanger,Heat Exchanger
Temperature /K,1073.15,1073.15,1073.15,1073.15,1073.15,1073.15,1073.15,1073.15,1073.15,1073.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pressure /Pa,101325,101325,101325,101325,101325,101325,101325,101325,101325,101325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mass Flow /kg/s,0.991709,1.00829,0.991709,1.00829,1,1,0.991709,1.00829,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Molar Flow /mol/s,34.4025,60.9191,34.4025,60.9191,34.6616,60.9186,34.4025,60.9191,34.6616,60.9186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
User-Defined LMTD Correction Factor /-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
Outlet Vapor Fraction (Stream 1) /-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
Outlet Vapor Fraction (Stream 2) /-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
Pressure Drop (Cold Side) /Pa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [36]:
# exergy destruction calculation using function

def edestruct(stream_in, stream_out, exergyin=False, exergyout=False):
    """
    Function for calculating exergy destruction of a component or system using exergy of streams
    stream_in:  list of streams going into the component 
    stream_out: list of streams going out of the component
    the format is (['stream_1','stream_2'],['stream_3','stream_4'])
    the result is in J/s
    
    to obtain the exergy in, use (['stream_1','stream_2'],['stream_3','stream_4'],exergyin=True)
    to obtain the exergy out, use (['stream_1','stream_2'],['stream_3','stream_4'],exergyout=True)
    
    WARNING: check the units of each property of the streams which are used for calling the dictionaries
    """
    E_in = 0
    E_out = 0
    for i in range(len(objects)):
        if objects[i].attrib['name'] in stream_in:
            if objects[i].attrib['type'] == 'Energy Stream':
                E_in = E_in + float(df[objects[i].attrib['name']]['Energy Flow /kW'])*1000 # J/s
            else:
                E_in = E_in + float(df[objects[i].attrib['name']]['Exergy of Stream /J/s'])
        if objects[i].attrib['name'] in stream_out:
            if objects[i].attrib['type'] == 'Energy Stream':
                E_out = E_out + float(df[objects[i].attrib['name']]['Energy Flow /kW'])*1000 # J/s
            else:
                E_out = E_out + float(df[objects[i].attrib['name']]['Exergy of Stream /J/s'])
        else:
            pass
    
    E_D = E_in - E_out
    if exergyin:
        return E_in
    elif exergyout:
        return E_out
    else:
        return E_D

In [38]:
# bonus: exergy balance check

def ebalance(stream_in, stream_out):
    """
    Function for checking the exergy balance of a component or system using exergy of streams
    stream_in:  list of streams going into the component 
    stream_out: list of streams going out of the component
    the format is (['stream_1','stream_2'],['stream_3','stream_4'])
    """
    
    E_Destruction = edestruct(stream_in, stream_out)
    E_in_tot = edestruct(stream_in, stream_out, exergyin=True)
    E_out_tot = edestruct(stream_in, stream_out, exergyout=True)

    if E_Destruction >= 0 and -0.1 < E_in_tot - E_out_tot - E_Destruction < 0.1:
        balance = True
    else:
        balance = False
    
    return balance

In [33]:
edestruct(['water_in','air_in','ESTR-01'],['water_out','air_out','ESTR-02'])

110553.09283901379

In [35]:
edestruct(['water_in','air_in','ESTR-01'],['water_out','air_out','ESTR-02'],exergyin=True)

2480405.599880904

In [39]:
ebalance(['water_in','air_in','ESTR-01'],['water_out','air_out','ESTR-02'])

True